# Resilient Terrestrial Sites
This is a draft indicator for the 2022 base blueprint.

Created by Amy Keister, modified by Blair Tirpak
last run by Blair Tirpak on February 24,2022

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"C:\Data\ResilientSites\TerResilient.gdb"

In [3]:
# define final outputs
SEout = r"C:\Data\ResilientSites\ResilientTerrestrialSites.tif"

In [4]:
# define sub-indicator outputs to help with user support

In [5]:
# define rasters used for cell size, extent, and snapping
# SE Extent with unapproved tribal land removed
SEraster= r"C:\Data\ResilientSites\SEextent2022_BIA.tif"

In [6]:
# define additional inputs
TerRUS= r"C:\Data\ResilientSites\Resilience_update_2021\resilience_update_2021.gdb\Resilience"


### Start Analysis

In [7]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

C:\Data\ResilientSites\TerResilient.gdb


### Relassify the terrestrial data

In [9]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask(TerRUS, SEraster); out_raster.save("TerRSE")

In [10]:
# reclassify the terrestrial data
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify("TerRSE", "Value", \
    "-3503 -3502 NODATA;-3501 -3500 0;-3499 -2000 1;-1999 -1000 2;-999 -500 3;-499 500 4;501 1000 5;1001 2000 6;2001 3500 7", "DATA"); \
    out_raster.save("TerRSERclss")

## Remove coastal extent

In [ ]:
# copy raster to snap to terra data
with arcpy.EnvManager(scratchWorkspace=r"C:\Data\ResilientSites\ResilientSites\Default.gdb", snapRaster="ResilientCoastalSites.tif", workspace=r"C:\Data\ResilientSites\ResilientSites\Default.gdb"):
    arcpy.management.CopyRaster("Coastal_data_extent", r"C:\Data\ResilientSites\TerResilient.gdb\Coastal", '', None, "127", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "GRID", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [12]:
# reclass coastal to ID in extent
out_raster = arcpy.sa.Reclassify("Coastal", "Value", "1 10;2 10", "NODATA"); 
out_raster.save("Coastal1cl")

In [ ]:
arcpy.MosaicToNewRaster_management("TerRSERclss;Coastal1cl", OutWorkspace,"TerCoastal", None, "8_BIT_UNSIGNED", None, 1, "Maximum","FIRST")

In [ ]:
remap = RemapValue([[10, "NODATA"], [7,7], [6,6], [5,5], [4,4], [3,3], [2,2], [1,1],[0,0]])
out_raster = arcpy.sa.Reclassify("TerCoastal", "Value", remap, "NODATA"); 
out_raster.save("Terra")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [ ]:
# set code block for next step

codeblock = """
def Reclass(v):
	if v == 7:
		return '7 = Most resilient'
	elif v == 6:
		return '6 = More resilient'
	elif v == 5:
		return '5 = Slightly more resilient'
	elif v == 4:
		return '4 = Average/median resilience'
	elif v == 3:
		return '3 = Slightly less resilient'
	elif v == 2:
		return '2 = Less resilient'
	elif v == 1:
		return '1 = Least resilient'
	elif v == 0:
		return '0 = Developed'
"""

In [ ]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("Terra", "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

In [ ]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 7:
		return 32
	if Value == 6:
		return 94
	if Value == 5:
		return 169
	if Value == 4:
		return 255
	if Value == 3:
		return 224
	if Value == 2:
		return 191
	if Value == 1:
		return 156
	if Value == 0:
		return 156
	else:
		return 255
		
def Reclass2(Value):
	if Value == 7:
		return 89
	if Value == 6:
		return 138
	if Value == 5:
		return 194
	if Value == 4:
		return 255
	if Value == 3:
		return 192
	if Value == 2:
		return 135
	if Value == 1:
		return 85
	if Value == 0:
		return 156
	else:
		return 255
		
def Reclass3(Value):
	if Value == 7:
		return 5
	if Value == 6:
		return 58
	if Value == 5:
		return 118
	if Value == 4:
		return 191
	if Value == 3:
		return 128
	if Value == 2:
		return 75
	if Value == 1:
		return 31
	if Value == 0:
		return 156
	else:
		return 255
"""

In [ ]:
# calculate Red field
arcpy.management.CalculateField("Terra", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("Terra", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("Terra", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

In [ ]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("Terra", SEraster); out_raster.save("TerFinal")

In [ ]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("TerFinal", SEout , '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

## END